In [1]:
import re
import pandas as pd
from datetime import datetime

df=pd.ExcelFile('./rawTestData.xlsx').parse(sheet_name=0, dtype=object, engine='xlrd', verbose=True)

Reading sheet 0


## - 사용할 컬럼명 저장 (array)

In [2]:
array=[]
for i in df:    
    array.append(df.iloc[0,df.columns.get_loc(i)])
array=array[:-2]

## - 엑셀에서 읽어온 dataframe 중 사용하는 컬럼만 추출

In [3]:
df_use=df.loc[1:,['센터코드','IRB승인 번호','환자대체번호','외과병리접수일자','외과병리판독일자','외과병리순번','검사결과내용']]

In [4]:
df_use

,센터코드,IRB승인 번호,환자대체번호,외과병리접수일자,외과병리판독일자,외과병리순번,검사결과내용
1,SNUBH,IRB,24281261,20150212,20150217,1,< 병리검사 의뢰서 입력내용 > \r1.채취부위:\r2.수 술 명:\r3.임상진단:...
2,SNUBH,IRB,24281300,20160118,20160122,1,< 병리검사 의뢰서 입력내용 > \r1.채취부위:\r2.수 술 명:\r3.임상진단:...
3,SNUBH,IRB,24299813,20150414,20150422,2,< 병리검사 의뢰서 입력내용 > \r1.채취부위:Rt. ovary\r2.수 술 명:...
4,SNUBH,IRB,24301235,20161201,20161212,1,< 병리검사 의뢰서 입력내용 > \r1.채취부위:\r2.수 술 명:\r3.임상진단:...
5,SNUBH,IRB,24303705,20150807,20150813,1,< 병리검사 의뢰서 입력내용 > \r1.채취부위:1. Lt breast2. Lt l...
...,...,...,...,...,...,...,...
8257,SNUBH,IRB,25848263,20190521,20190528,1,< 병리검사 의뢰서 입력내용 > \r1.채취부위:\r2.수 술 명:\r3.임상진단:...
8258,SNUBH,IRB,25860009,20160404,20160408,1,< 병리검사 의뢰서 입력내용 > \r1.채취부위:1. Rt. breast mass\...
8259,SNUBH,IRB,25896077,20190404,20190410,1,< 병리검사 의뢰서 입력내용 > \r1.채취부위:Lt. breast\r2.수 술 명...
8260,SNUBH,IRB,25900875,20160901,20160907,1,< 병리검사 의뢰서 입력내용 > \r1.채취부위:\r2.수 술 명:\r3.임상진단:...


In [5]:
len(df_use)

8261

## - 데이터를 저장할 dataframe 생성

In [6]:
df_result=pd.DataFrame(index=range(0,8261),columns=array)

In [7]:
print(df_result)

CENTER_CD IRB_APRV_NO PT_SBST_NO SGPT_ACPT_YMD SGPT_READ_YMD SGPT_SEQ  \
0          NaN         NaN        NaN           NaN           NaN      NaN   
1          NaN         NaN        NaN           NaN           NaN      NaN   
2          NaN         NaN        NaN           NaN           NaN      NaN   
3          NaN         NaN        NaN           NaN           NaN      NaN   
4          NaN         NaN        NaN           NaN           NaN      NaN   
...        ...         ...        ...           ...           ...      ...   
8256       NaN         NaN        NaN           NaN           NaN      NaN   
8257       NaN         NaN        NaN           NaN           NaN      NaN   
8258       NaN         NaN        NaN           NaN           NaN      NaN   
8259       NaN         NaN        NaN           NaN           NaN      NaN   
8260       NaN         NaN        NaN           NaN           NaN      NaN   

     SGPT_MAIN_HVST_SITE_CD SGPT_MAIN_HVST_SITE_NM SGPT_OPRT_LOCA_CO

## - 사용할 정규식들 정의

In [26]:
find=[]                 #7~40번까지 정규식 저장하는 array

#7~11
find.append(re.compile("(< 병리검사 의뢰서 입력내용 >)"))
find.append(re.compile("(< 병리검사 의뢰서 입력내용 >)"))
find.append(re.compile("(< 병리검사 의뢰서 입력내용 >)"))
find.append(re.compile("Size\sof\stumor\s\(invasive\scomponent\sonly\):\s(\d+\.?\d*)\sx\s(\d+\.?\d*)\sx\s(\d+\.?\d*)cm"))
find.append(re.compile("(MRI|Somo)"))

#12~16
find.append(re.compile("Extent\sof\stumor\s*:\s*(\d+\.?\d*)\sx\s(\d+\.?\d*)\sx\s(\d+\.?\d*)cm"))
find.append(re.compile("(< 병리검사 의뢰서 입력내용 >)"))
find.append(re.compile("Surgical\smargins\s*:\s*clear\s*but\s*close\s*to\s*(.*)[safety|Safety]|Surgical\smargins\s*:\s*involvement\s*of\s*(.+)\([safety|Safety]"))
find.append(re.compile("(< 병리검사 의뢰서 입력내용 >)"))
find.append(re.compile("Histologic grade\s*\(.*\):\s*Grade\s*(\w+//*\w*)"))

#17~21
find.append(re.compile("(< 병리검사 의뢰서 입력내용 >)"))
find.append(re.compile("nuclear\sgrade:\sgrade\s(\d+)"))
find.append(re.compile("necrosis:\s(\w+)"))
find.append(re.compile("(< 병리검사 의뢰서 입력내용 >)"))
find.append(re.compile("extensive\sintraductal\scomponent\s*:\s*(\w+)"))
#22~26
find.append(re.compile("involvement\sof\slactiferous\sduct\s*:\s*(\w+)"))
find.append(re.compile("(< 병리검사 의뢰서 입력내용 >)"))
find.append(re.compile("(< 병리검사 의뢰서 입력내용 >)"))
find.append(re.compile("(< 병리검사 의뢰서 입력내용 >)"))
find.append(re.compile("(< 병리검사 의뢰서 입력내용 >)"))
#27~31
find.append(re.compile("(< 병리검사 의뢰서 입력내용 >)"))
find.append(re.compile("(< 병리검사 의뢰서 입력내용 >)"))
find.append(re.compile("(< 병리검사 의뢰서 입력내용 >)"))
find.append(re.compile("(< 병리검사 의뢰서 입력내용 >)"))
find.append(re.compile("(< 병리검사 의뢰서 입력내용 >)"))
#32~36
find.append(re.compile("(< 병리검사 의뢰서 입력내용 >)"))
find.append(re.compile("(< 병리검사 의뢰서 입력내용 >)"))
find.append(re.compile("(< 병리검사 의뢰서 입력내용 >)"))
find.append(re.compile("Microcal\w+:\s(\w+)"))
find.append(re.compile("(< 병리검사 의뢰서 입력내용 >)"))
#37~39
find.append(re.compile("pT(\w)\wN\d"))
find.append(re.compile("pT\w\wN(\d)"))
find.append(re.compile("(< 병리검사 의뢰서 입력내용 >)"))

len(find)

33

In [27]:
for i in range(0,len(df_use)):
    for col in range(0,6):
        df_result.iloc[i,col]=df_use.iloc[i,col]

    for col in range(6,39):
        df_result.iloc[i,col]=find[col-6].findall(df_use.iloc[i,df_use.columns.get_loc('검사결과내용')])
    df_result.iloc[i,39]=datetime.now()
    

In [28]:
df_result['SGPT_NIPL_INVA_CONT']


0                     []
1               [absent]
2                     []
3                     []
4               [absent]
              ...       
8256    [absent, absent]
8257                  []
8258            [absent]
8259            [absent]
8260                  []
Name: SGPT_NIPL_INVA_CONT, Length: 8261, dtype: object

In [22]:
df_result.to_csv('result.csv',sep=',',header=True,index=False, encoding='utf-8-sig')